In [ ]:
# Some manual analysis of embedding done during MTM marathon 2018
# to generate embedding used file emb_gen.sh

In [ ]:
import numpy as np

In [ ]:
def read_avg_embed(filename):
    emb_p=[]
    emb_p_idx=[]
    emb_avg = []
    embed_file=open(filename,'rt')
    prev_sent_num=-1
    for l in embed_file:
        l1=l.split(' ')
        sent_num = l1[0].split('T')[0].split('S')[1]
        emb_this = np.array([float(i) for i in str(l1[1]).split(',')])
        if (sent_num==prev_sent_num):
            cnt+=1
            emb_avg[-1]=emb_avg[-1]+emb_this            
        else:
            if (len(emb_avg)>0):
                emb_avg[-1]/=(cnt+1)
            cnt=0
            emb_avg.append(emb_this)
            prev_sent_num=sent_num
        emb_p_idx.append(l1[0])
        emb_p.append(emb_this)
    #processing the last
    emb_avg[-1]/=(cnt+1)
    
    embed_file.close()
    #return [np.array(emb_p_idx), np.array(emb_p, dtype=float), np.array(emb_avg)]
    return np.array(emb_avg)

In [ ]:
def read_first_embed(filename):
    emb_p=[]
    emb_avg = []
    embed_file=open(filename,'rt')
    prev_sent_num=-1
    for l in embed_file:
        l1=l.split(' ')
        sent_num = l1[0].split('T')[0].split('S')[1]
        emb_this = np.array([float(i) for i in str(l1[1]).split(',')])
        if (sent_num!=prev_sent_num):
            prev_sent_num=sent_num
            emb_p.append(emb_this)
    embed_file.close()
    return np.array(emb_p)

In [ ]:
def read_each_embed(filename):
    emb_p=[]
    emb_p_idx=[]
    embed_file=open(filename,'rt')
    prev_sent_num=-1
    for l in embed_file:
        l1=l.split(' ')
        sent_num = l1[0].split('T')[0].split('S')[1]
        emb_this = np.array([float(i) for i in str(l1[1]).split(',')])
        emb_p_idx.append(l1[0])
        emb_p.append(emb_this)

    embed_file.close()
    #return [np.array(emb_p_idx), np.array(emb_p, dtype=float)]
    return np.array(emb_p, dtype=float)

In [ ]:
def read_emb(filename, emb_type):
    return {
        'avg':lambda x:read_avg_embed(x),
        'first': lambda x: read_first_embed(x),
        'each': lambda x: read_each_embed(x)
    }[emb_type](filename)

In [ ]:
def cosine_sim(A):
    similarity = np.dot(A, A.T)
    square_mag = np.diag(similarity)
    inv_square_mag = 1 / square_mag
    inv_square_mag[np.isinf(inv_square_mag)] = 0
    inv_mag = np.sqrt(inv_square_mag)

    cosine = similarity * inv_mag
    cosine = cosine.T * inv_mag
    return cosine
    

In [ ]:
langs = ['ar','de', 'fa', 'ko', 'ru', 'zh']

In [ ]:
# types = {'context': ['avg', 'first', 'each'],
#          'hidden':['avg', 'first' ]}
types = {'hidden': ['avg', 'first'],
         'context':['avg', 'first' ]}

In [ ]:
means={}
N3 = {}
for t in types:
    means[t]={}
    N3[t]={}
    for t2 in types[t]:
        means[t][t2]={}
        N3[t][t2]={}
        emb = {}
        for l in langs:
            filename = 'outdir/ted.%s-en.test.bpe.%s_%s'%(l,l,t)
            print('loading %s with type = %s,%s...'%(filename,t,t2))
            emb[l] = read_emb(filename,t2)
        for l in langs:
            means[t][t2][l] = {}
            N3[t][t2][l]={}
            for l2 in langs:
#                if (l!=l2):
                    all_emb = np.append(emb[l], emb[l2], axis = 0)
                    sim_matrix = cosine_sim(all_emb)
                    avgs_d = []
                    size = np.shape(emb[l])[0]
                    for i in range(size):
                        avgs_d.append(sim_matrix[i,i+size])
                    means[t][t2][l][l2] = np.mean(np.array(avgs_d))
                    print("%s %s %s %s"%(t,t2,l,l2))
                    print(means[t][t2][l][l2])
                    cnt1=0
                    cnt3=0
                    sim_nums =  np.argsort(sim_matrix, axis=1).T
                    nn3 = sim_nums[-4:]
                    for i in range(size):
                        if (nn3[2][i]==i+size):
                            cnt1+=1
                        for j in range(3):
                            if (nn3[j][i]==i+size):
                                cnt3+=1
                    N3[t][t2][l][l2] = cnt3/size
                    print(cnt1/size)
                    print(N3[t][t2][l][l2])
                                    
                
                

In [ ]:
np.mean(sim_matrix[0:2*size,0:2*size])

In [ ]:
sim_nums =  np.argsort(sim_matrix, axis=1).T
nn3 = sim_nums[-4:]

In [ ]:
## Prining matrices for report
for l in langs:
    s=''
    for l2 in langs:
        #s+='%.2f'%(means['hidden']['avg'][l][l2]*100)
        s+='%.2f'%(N3['context']['first'][l][l2]*100)
        s+=','
    print(s)

In [ ]:
cnt1=0
cnt3=0
for i in range(size):
    if (nn3[2][i]==i+size):
        cnt1+=1
    for j in range(3):
        if (nn3[j][i]==i+size):
            cnt3+=1
print(cnt1/size)
print(cnt3/size)

In [ ]:
ar_en_context_first = read_first_embed('outdir/ted.ar-en.dev.bpe.ar_context')
de_en_context_first = read_first_embed('outdir/ted.de-en.dev.bpe.de_context')

In [ ]:
# word context level based similarity
test_mx_ar_vs_de = np.append(ar_en_context[0:124],de_en_context[0:124], axis=0)
sim_matrix_ar_vs_de_context=cosine_sim(test_mx_ar_vs_de)

In [ ]:
# similarity of english to deutch
[np.mean(sim_matrix_ar_vs_de_context[0:124,124:248]) , 
np.std(sim_matrix_ar_vs_de_context[0:124,124:248]) ]

In [ ]:
# similarity of deutch to deutch 
[np.mean(np.abs(sim_matrix_ar_vs_de_context[0:124,0:124])),
 np.std(np.abs(sim_matrix_ar_vs_de_context[0:124,0:124]))]

In [ ]:
avgs_d = []
for i in range(124):
    avgs_d.append(sim_matrix_ar_vs_de_context[i,i+124])
[np.mean(np.array(avgs_d)),np.std(np.array(avgs_d))]

In [ ]:
# avg context vector based similarities


test_mx_seq_avg = np.append(ar_en_context_avg,de_en_context_avg, axis=0)

In [ ]:
sim_matrix_seq_avg=cosine_sim(test_mx_seq_avg)

In [ ]:
[np.mean(sim_matrix_seq_avg[0:1958,1958:3916]) , 
np.std(sim_matrix_seq_avg[0:1958,1958:3916])]

In [ ]:
[np.mean(sim_matrix_seq_avg[0:1958,0:1958]) , 
np.std(sim_matrix_seq_avg[0:1958,0:1958])]

In [ ]:
avgs_h = []
for i in range(1958):
    avgs_h.append(sim_matrix_seq_avg[i,i+1958])

In [ ]:
[np.mean(np.array(avgs_h)),np.std(np.array(avgs_h))]

In [ ]:
# difference vector (avegare over first 40K context vectors)
diff_avg_vector = np.average(de_en_context[0:40825]-ar_en_context[0:40825], axis=0)

In [ ]:
supposed = de_en_context[0:1000]+diff_avg_vector

In [ ]:
actual =  ar_en_context[0:1000]

In [ ]:
sim_mx = np.append(supposed,actual, axis = 0)

In [ ]:
sim_mx = cosine_sim(sim_mx)

In [ ]:
avgs_sh = []
for i in range(1000):
    avgs_sh.append(sim_mx[i,i+1000])
[np.mean(np.array(avgs_sh)),np.std(np.array(avgs_sh))]

In [ ]:
sim_nums =  np.argsort(sim_matrix_seq_avg, axis=1).T

In [ ]:
nn3 = sim_nums[-4:]

In [ ]:
cnt=0
for i in range(1958):
    if (nn3[3][i]==i):
        cnt+=1
cnt

In [ ]:
cnt=0
for i in range(1958):
    if (nn3[0][i]==i+1958):
        cnt+=1
    if (nn3[2][i]==i+1958):
        cnt+=1        
    if (nn3[1][i]==i+1958):
        cnt+=1        
cnt/1958.0

In [ ]:
cnt=0
for i in range(1958):
    if (nn3[0][i]==i+1958):
        cnt+=1
cnt/1958.0

In [ ]:
test_mx_seq_first = np.append(ar_en_context_first,de_en_context_first, axis=0)

In [ ]:
sim_matrix_seq_first=cosine_sim(test_mx_seq_first)

In [ ]:
sim_nums_first =  np.argsort(sim_matrix_seq_first, axis=1).T

In [ ]:
nn3 = sim_nums_first[-4:]